In [39]:
import pandas as pd
import requests

In [40]:
provinces = [
  {"province_id": 663494, "province_name": "Drenthe"},
  {"province_id": 663607, "province_name": "Flevoland"},
  {"province_id": 663423, "province_name": "Fryslân"},
  {"province_id": 663687, "province_name": "Gelderland"},
  {"province_id": 663343, "province_name": "Groningen"},
  {"province_id": 664144, "province_name": "Limburg "},
  {"province_id": 664173, "province_name": "Niet-provinciaal ingedeeld"},
  {"province_id": 664062, "province_name": "Noord-Brabant"},
  {"province_id": 663839, "province_name": "Noord-Holland "},
  {"province_id": 663577, "province_name": "Overijssel"},
  {"province_id": 663758, "province_name": "Utrecht"},
  {"province_id": 663989, "province_name": "Zeeland"},
  {"province_id": 663912, "province_name": "Zuid-Holland"}
]

In [41]:
date = "20170315"

In [45]:
headers = {"User-Agent": "Mozilla/5.0"}
provinces = requests.get(f"https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/TK{date}", headers=headers)
provinces = [{"province_id": x["Id"], "province_name": x["Value"]} for x in provinces.json()["Regios"]["Regios"][0]["Options"][1:]]

In [46]:
output = []
for province in provinces:
  regions = requests.get(f"https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/TK{date}/{province['province_id']}", headers=headers)
  districts = [x["Id"] for x in regions.json()["Regios"]["Regios"][1]["Options"][:1]]
  parties = [{"party_id": x["KandidatenlijstId"], "party": x["Naam"]} for x in regions.json()["Stemregio"]["Partij"]]

  for district in districts:
    for party in parties:
      r = requests.get(
        f"https://www.verkiezingsuitslagen.nl/verkiezingen/KandidaatJson/TK{date}?stemregioId={district}&kandidatenLijst={party['party_id']}",
        headers=headers
      )

      frag = pd.DataFrame(r.json()["KandidaatUitslag"])
      frag["party_id"] = party['party_id']
      frag["party"] = party['party']
      frag["province_id"] = province['province_id']
      output.append(frag)

In [47]:
df = pd.concat(output, ignore_index=True)
df = df.rename(columns={
  "Naam": "candidate",
  "Gekozen": "elected",
  "AantalStemmen": "value"
})
df = df[["candidate", "elected", "value", "party", "party_id", "province_id"]].rename(columns={"province_id": "district_id"})
df["value"] = df["value"].str.replace(".", "").astype(int)

df.to_csv(f"../data_output/Netherlands/{date[0:4]}_camera_district.csv.gz", index=False, compression="gzip")

df.sort_values("value")

KeyError: "['candidate', 'elected', 'value'] not in index"

In [38]:
# https://www.verkiezingsuitslagen.nl/verkiezingen/detailJson/TK20120912/663494